In [52]:
import time

In [14]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [15]:
genome = readGenome('sources/phix.fa')

In [10]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

In [44]:
def naive2(p, t):
    occurrences = []
    rangeEval = len(t) - len(p) + 1
    i = 0
    while i < rangeEval:
        match = True
        for j in range(len(p)):
            try:
                if t[i+j] != p[j]:
                    match = False
                    break
            except:
                return 'error at',i+j
        if match:
            occurrences.append(i)
            i += len(p)
        else:
            i += 1
    return occurrences

In [25]:
t = genome
p = phix_reads[2]
naive2(p, t)

[]

In [38]:
i=0
for read in phix_reads:
    try:
        if len(naive2(read,genome))>0:
            print(i,naive2(read,genome))
    except: 
        print('failed at ',i)
    i += 1

failed at  26
failed at  30
failed at  101
failed at  182
failed at  190
failed at  203
failed at  205
failed at  230
failed at  241
242 [29]
failed at  264
failed at  300
failed at  356
369 [23]
failed at  493
511 [10]
failed at  520
failed at  575
failed at  598
failed at  728
749 [21]
762 [2]
817 [20]
failed at  828
835 [61]
failed at  851
failed at  875
failed at  988


In [45]:
i=0
for read in phix_reads:
    if len(naive2(read,genome))>0:
        print(i,naive2(read,genome))
    i += 1

242 [29]
369 [23]
511 [10]
749 [21]
762 [2]
817 [20]
835 [61]


In [43]:
len(genome)

5386

In [6]:
import random
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

In [7]:
# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# Count how many reads match the genome exactly
numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


In [6]:
!curl http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq -o 'sources/ERR266411_1.first1000.fastq'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  248k  100  248k    0     0   116k      0  0:00:02  0:00:02 --:--:--  116k


In [4]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

In [8]:
import collections
phix_reads = readFastq('sources/ERR266411_1.first1000.fastq')
count = collections.Counter()
for read in phix_reads:
    count.update(read)
count

Counter({'T': 30531, 'A': 28426, 'C': 21890, 'G': 19147, 'N': 6})

In [23]:
len(phix_reads)

1000

In [60]:
start_time = time.time()
numMatched = 0
n = 0
for r in phix_reads:
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))
print("--- %s seconds ---" % (time.time() - start_time))

7 / 1000 reads matched the genome exactly!
--- 3.7295589447021484 seconds ---


In [48]:
# Now let's try matching just the first 30 bases of each read
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive2(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

459 / 1000 reads matched the genome exactly!


In [49]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [56]:
start_time = time.time()
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))
print("--- %s seconds ---" % (time.time() - start_time))

932 / 1000 reads matched the genome exactly!
--- 6.798330783843994 seconds ---
